In [30]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import mnist

In [31]:
# from PIL import Image
# import glob
# import os

# # new folder path (may need to alter for Windows OS)
# # change path to your path
# path = '/Users/tareklein/Downloads/archive/train/plus cleaned' #the path where to save resized images
# # create new folder
# if not os.path.exists(path):
#     os.makedirs(path)

# # loop over existing images and resize
# # change path to your path
# for filename in glob.glob(path + '/*.jpg'): #path of raw images
#     img = Image.open(filename).resize((28,28))
#     # save resized images to new folder with existing filename
#     img.save('{}{}{}'.format(path,'/',os.path.split(filename)[1]))

In [32]:
def normalize_img(image, label):
    return (tf.cast(image, tf.float32) / 255.0, label)

In [33]:

mnist_corrupted = 'mnist_corrupted'
mnist = 'mnist'

dataset = mnist

(train_dataset, test_dataset), ds_info = tfds.load(
    dataset,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
    )

# (train_dataset2, test_dataset2), ds_info = tfds.load(
#     mnist_corrupted,
#     split=['train', 'test'],
#     shuffle_files=True,
#     as_supervised=True,
#     with_info=True
#     )

In [34]:
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

train_dataset = train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Store dataset in cache
train_dataset = train_dataset.cache()
# Shuffle data
train_dataset = train_dataset.shuffle(len(train_dataset))
# Split up into a batch of 64
train_dataset = train_dataset.batch(64)
# Grab other stuff before it done dealing with current stuff
# Optimization 
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)


In [35]:
#test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

test_dataset = test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Split up into a batch of 64
test_dataset = test_dataset.batch(64)
# Store dataset in cache
test_dataset = test_dataset.cache()
# Grab other stuff before it done dealing with current stuff
# Optimization 
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [36]:
# Visualise image tensor (batch has to be 1)
visualise = False
if visualise: 
    for (img, label) in train_dataset:
        print(img.numpy(), label.numpy())
        break
    else:
        pass

In [37]:
train = False
if train:
    model = tf.keras.models.Sequential([
        tf.keras.Input((28,28,1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

    model.fit(train_dataset, epochs=100, validation_data=(train_dataset))


    model.save('handwritten.model')
else:
    pass
#model.summary()

Epoch 1/100
469/469 [==============================] - 3s 3ms/step - loss: 0.2578 - accuracy: 0.9255 - val_loss: 0.1089 - val_accuracy: 0.9669
Epoch 2/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0955 - accuracy: 0.9715 - val_loss: 0.0673 - val_accuracy: 0.9797
Epoch 3/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0615 - accuracy: 0.9804 - val_loss: 0.0433 - val_accuracy: 0.9868
Epoch 4/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0435 - accuracy: 0.9864 - val_loss: 0.0304 - val_accuracy: 0.9911
Epoch 5/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0335 - accuracy: 0.9895 - val_loss: 0.0231 - val_accuracy: 0.9929
Epoch 6/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0264 - accuracy: 0.9916 - val_loss: 0.0294 - val_accuracy: 0.9902
Epoch 7/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0226 - accuracy: 0.9928 - val_loss: 0.0127 - val_accuracy: 0.9961

INFO:tensorflow:Assets written to: handwritten.model/assets


In [38]:
model = tf.keras.models.load_model('handwritten.model')

loss, accuracy = model.evaluate(train_dataset)

print(loss)
print(accuracy)

469/469 [==============================] - 1s 926us/step - loss: 3.0180e-09 - accuracy: 1.0000
3.017981642372547e-09
1.0


In [39]:
# from matplotlib.lines import lineStyles
# import matplotlib.pyplot as plt
# plt.figure(figsize=(18,8))
# plt.plot(0,0, history.history['accuracy'], label='Accuracy', lw=3)
# plt.plot(0,0, history.history['loss'], label='Loss', lw=3, linestyle='--')
# plt.legend


In [40]:
def rescaleFrame(frame, scale):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[1] * scale)
    print(width, height)
    dimensions = (width, height)
    
    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [41]:
capture = cv.VideoCapture(0)
scale = 0.50
number = []
while True:
    isTrue, frame = capture.read()
    frame_resized = rescaleFrame(frame, scale)

    # Grayscaled
    gray = cv.cvtColor(frame_resized, cv.COLOR_BGR2GRAY)
    gray_28 = cv.resize(gray, (28,28), interpolation=cv.INTER_AREA)
    
    gray_28 = gray_28.astype("float32") / 255.0
    gray_28 = np.expand_dims(gray_28, axis=-1)
    
    prediction = model.predict(gray_28.reshape(1,28,28))
    predargmax = np.argmax(prediction)

    # Black and white
    range = (150, 255)
    #range = (125, 255)
    ret, thresh = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    # 28
    ret_28, thresh_28 = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    thresh_28 = cv.resize(thresh_28, (28,28), interpolation=cv.INTER_AREA)
    contours_28, hierarchies_28 = cv.findContours(thresh_28, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    thresh_28 = thresh_28.astype("float32") / 255.0
    thresh_28 = np.expand_dims(thresh_28, axis=-1)
    
    
    prediction_thresh_28 = model.predict(thresh_28.reshape(1,28,28))
    predargmax_thresh_28 = np.argmax(prediction_thresh_28)


    cv.putText(gray, 'Gray: ' + str(predargmax), (20,40), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv.putText(gray, 'B&W: ' + str(predargmax_thresh_28), (20,80), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
    if len(number) == 1:
        cv.putText(gray, str(number[0]) + ' + ', (20,120), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 4)
    if len(number) >= 2:
        cv.putText(gray, str(number[0]) + ' + ' + str(number[1]) + ' = ' + str(number[0] + number[1]), (20,120), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 4)
        
            
    cv.imshow('Gray scale', gray)
    cv.imshow('Black & White (gray threshold)', thresh)
    #cv.imshow('Black & White (gray threshold) 28x28', thresh_28)
    
    cv.moveWindow('Black & White (gray threshold)', cv.getWindowImageRect('Gray scale')[2], 0)
    cv.moveWindow('Black & White (gray threshold) 28x28', cv.getWindowImageRect('Gray scale')[2], 0)
   

    if cv.waitKey(10) & 0xFF==ord('a'):
        number.append(predargmax_thresh_28)
    if cv.waitKey(10) & 0xFF==ord('q'):
        break
    
capture.release()
cv.waitKey(50)
cv.destroyAllWindows()
cv.waitKey(50)

640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 7ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 7ms/step
640 640
1/1 [==============================] - 0s 7ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/st

-1

: 